# December 10, 2021

https://adventofcode.com/2021/day/10

In [1]:
import pandas as pd
import numpy as np
from collections import deque

In [2]:
with open("../data/2021/10.txt", "r") as f:
    data = f.read()

data = data.split("\n")

In [3]:
test = '''[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]'''

test = test.split("\n")

# Part 1

In [4]:
# I think reading right-to-left is faster to detect mal-formatted lines
# because an incomplete line is known as soon as you find an opener and there are no closers stacked
# This means there's no need to read the entire line!
# but going left-to-right is needed to detect the FIRST corruption.

def scan_line( line, verbose = False ):
    match_dict = {"[":"]", "{":"}", "<":">", "(":")"}

    lefts = deque()

    for pos, char in enumerate(line):
        if char in match_dict.keys():
            lefts.append(char)
        elif char in match_dict.values():
            if len(lefts) == 0:
                if verbose:
                    print(f'''At pos {pos}: Found unmatched close {char}''')
                return "corrupt", char
            
            left = lefts.pop()
            if char != match_dict[ left ]:
                if verbose:
                    print(f'''At pos {pos}: Found {char}, expected {match_dict[left]}''')
                return "corrupt", char
                
    # incomplete!
    if len(lefts) > 0:
        return "incomplete", lefts
    
    # yay!
    return "okay", None

def score_line( line, verbose = False ):
    score_dict = {")": 3, "]": 57, "}": 1197, ">": 25137}
    result, details = scan_line( line, verbose )
    if result == "corrupt":
        score = score_dict[details]
    else:
        score = 0

    return score

def score_text( text ):
    score = 0
    for t in text:
        score += score_line( t )
    return score

In [5]:
for l in test:
    x = scan_line(l, True)
    if x is True: print(x)
    if x is False: print(x)

At pos 12: Found }, expected ]
At pos 8: Found ), expected ]
At pos 7: Found ], expected )
At pos 10: Found ), expected >
At pos 16: Found >, expected ]


In [6]:
for l in test:
    print(score_line(l, False))

0
0
1197
0
3
57
0
3
25137
0


In [7]:
score_text(test)

26397

In [8]:
score_text(data)

436497

# Part 2

In [9]:
# Let's be wacky. We can write our own += operator
# to add autocomplete bounties across a line
class AutoCompleteScore():
    def __init__(self, x=0):
        self.value = x
        
    def __iadd__(self, char):
        score_dict = {"(":1, "[":2, "{":3, "<":4}

        if char in score_dict.keys():
            return AutoCompleteScore( self.value*5 + score_dict[char] )
        else:
            return self

In [10]:
def score_ac( line ):
    ACS = AutoCompleteScore()

    result, details = scan_line( line )
    if result == "incomplete":
        while len(details) > 0:
            ACS += details.pop()

    return ACS.value

def score_text_ac( text ):
    scores = [ score_ac(line) for line in text ]
    scores = [x for x in scores if x != 0 ]
    scores.sort()

    return scores[ int(len(scores)/2) ]

In [11]:
for line in test:
    print(score_ac(line))

288957
5566
0
1480781
0
0
995444
0
0
294


In [12]:
scores = [ score_ac(line) for line in test ]

In [13]:
scores

[288957, 5566, 0, 1480781, 0, 0, 995444, 0, 0, 294]

In [14]:
score_text_ac(test)

288957

In [15]:
score_text_ac(data)

2377613374

In [16]:
score_text_ac(data)

2377613374